## NumPy and Vectorization

*[Coding along with Python for Algorithmic Trading, Yves Hilpisch, O'Reilly, 1st edition November 2020, ISBN-13: 978-1492053354; Quant Platform at [https://py4at.pqp.io](https://py4at.pqp.io)]*

In [1]:
# simulation with 1.000.000 repetitions done with pure Python
# computing Euler discretization of the geometric Brownian motion
# Euler discretization is a numerical method used to approximate the solution of ordinary differential equations (ODEs) 
# and stochastic differential equations (SDEs). It is a simple and widely used technique, named after Leonhard Euler, 
# who first proposed it.
import random
from math import exp, sqrt

In [2]:
S0 = 100
r = 0.05
T = 1.0
sigma = 0.2

values = []

repetitions = 1000000

In [3]:
%time
for _ in range(repetitions):
    ST = S0 * exp((r - 0.5 * sigma ** 2) * T + sigma * random.gauss(0, 1) * sqrt(T))
    values.append(ST)

CPU times: user 9 μs, sys: 4 μs, total: 13 μs
Wall time: 366 μs


In [4]:
# Numpy approach with vectorization
import numpy as np

In [5]:
%time
# simulating all the values with a single line of NumPy code and storing them in a ndarray object
ST_np = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.random.standard_normal(repetitions) * np.sqrt(T))

CPU times: user 1 μs, sys: 0 ns, total: 1 μs
Wall time: 2.86 μs


## pandas the DataFrame Class

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
# https://www.squash.io/how-to-use-matplotlib-inline-in-python/
%matplotlib inline
from pylab import mpl, plt
plt.style.use("seaborn-v0_8")
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif' # imports and configures the plotting package

In [7]:
# importing configparser, reading credentials
c = configparser.ConfigParser()
c.read('../../../../../../../tmp/pyalgo/pyalgo.cfg') # yeah, that's a path in my local file system
ch0 = c.get('quandl','api_key')
print("Don't be a fool and send your api key to GitHub")

NameError: name 'configparser' is not defined

In [ ]:
# Nasdaq Data Link Documentation https://docs.data.nasdaq.com/
# https://github.com/quandl/quandl-python
# import quandl # importing quandl wrapper
# quandl.ApiConfig.api_key = ch0 # providing api key
# quandl.ApiConfig.verify_ssl = False
# additional library to suppress "InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.nasdaq.com'"
# import urllib3
# urllib3.disable_warnings()

### replaced with nasdaq-data-link

In [ ]:
import nasdaqdatalink as nq
nq.ApiConfig.api_key = ch0 # providing api key
nq.ApiConfig.verify_ssl = False
# additional library to suppress "InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.nasdaq.com'"
import urllib3
urllib3.disable_warnings()

In [ ]:
# https://docs.data.nasdaq.com/docs/python-tables
# Example 1: Download data for Nokia (compnumber=39102)
data = nasdaqdatalink.get_table('MER/F1', compnumber="39102", paginate=True)
data.head()

In [ ]:
# Download an entire table
data = nasdaqdatalink.get_table('MER/F1', paginate=True)
data.head()

In [ ]:
# back to the bitcoin example
# Bitcoin data insights: https://data.nasdaq.com/databases/BCHAIN#quick-start-examples
btc_data = nasdaqdatalink.get_table('QDL/BCHAIN', paginate=True, code="MKPRU")

In [ ]:
btc_data

In [ ]:
btc_data.shape

In [ ]:
btc_data.info()

In [ ]:
# calculating a simple moving average
# https://www.geeksforgeeks.org/how-to-calculate-moving-average-in-a-pandas-dataframe/
# extract 'value' column and convert to DataFrame
btc_value = btc_data['value'].to_frame()
# calculate 100-day Simple Moving Average (SMA)
btc_value['SMA100'] = btc_value['value'].rolling(100).mean()
# remove NULL values
btc_value.dropna(inplace=True)
# Print DataFrame
btc_value

In [ ]:
# Plotting Simple Moving Averages
# plotting close price (value) and simple moving average of 30 days using .plot() method
btc_value[['value', 'SMA100']].plot(label='BTC/USD exchange rate', figsize=(16, 8)) # doesn't look right

In [ ]:
# calculation according to the example from Hilpisch's book
btc_data.loc['2013-10-10':].plot(title='BTC/USD exchange rate', figsize=(10, 6)) # doesn't look right either

In [ ]:
# examples with date: different days
btc_data_date1 = nasdaqdatalink.get_table('QDL/BCHAIN', paginate=True, code="MKPRU", date="2016-07-17,2016-07-16,2016-07-15")
btc_data_date1

In [ ]:
# examples with date: single day
btc_data_date2 = nasdaqdatalink.get_table('QDL/BCHAIN', paginate=True, code="MKPRU", date="2018-07-17")
btc_data_date2

In [ ]:
# examples with date: range between days
btc_data_date3 = nasdaqdatalink.get_table('QDL/BCHAIN', paginate=True, code="MKPRU", date = { 'gte': '2016-01-01', 'lte': '2016-12-31' })
btc_data_date3